# Introduction to Quasi-Monte Carlo Sequences

This notebook demonstrates the quasi-random sequence generators implemented in the `qmc_options` package.

In [ ]:
import numpy as np
import matplotlib.pyplot as plt
from qmc_options import generators

%matplotlib inline
plt.style.use('seaborn-v0_8-darkgrid')

## Van der Corput Sequence

The Van der Corput sequence is a 1-dimensional low-discrepancy sequence.

In [ ]:
# Generate Van der Corput sequence in base 2
N = 100
vdc_seq = np.array([generators.van_der_corput(2, i+1) for i in range(N)])

plt.figure(figsize=(12, 4))
plt.plot(vdc_seq, 'o-', markersize=3)
plt.xlabel('Index')
plt.ylabel('Value')
plt.title('Van der Corput Sequence (base 2)')
plt.grid(True, alpha=0.3)
plt.show()

## Halton Sequence

The Halton sequence extends Van der Corput to multiple dimensions using different prime bases.

In [ ]:
# Generate 2D Halton sequence
N = 500
halton_seq = generators.halton([2, 3], N)

# Compare with pure random
np.random.seed(42)
random_seq = np.random.rand(N, 2)

fig, (ax1, ax2) = plt.subplots(1, 2, figsize=(14, 6))

ax1.scatter(halton_seq[:, 0], halton_seq[:, 1], s=10, alpha=0.6)
ax1.set_title('Halton Sequence (2D)')
ax1.set_xlabel('Dimension 1')
ax1.set_ylabel('Dimension 2')
ax1.grid(True, alpha=0.3)

ax2.scatter(random_seq[:, 0], random_seq[:, 1], s=10, alpha=0.6)
ax2.set_title('Pseudo-Random (2D)')
ax2.set_xlabel('Dimension 1')
ax2.set_ylabel('Dimension 2')
ax2.grid(True, alpha=0.3)

plt.tight_layout()
plt.show()

print("Notice how Halton points are more uniformly distributed!")

## Good Lattice Points (GLP)

Good Lattice Points use Fibonacci numbers to create highly uniform point sets.

In [ ]:
# Generate GLP with different Fibonacci indices
fig, axes = plt.subplots(2, 2, figsize=(12, 12))
axes = axes.flatten()

for idx, m in enumerate([6, 8, 10, 12]):
    glp = generators.good_lattice_points(m)
    N = len(glp)
    
    axes[idx].scatter(glp[:, 0], glp[:, 1], s=20, alpha=0.6)
    axes[idx].set_title(f'GLP with m={m} (N={N} points)')
    axes[idx].set_xlabel('Dimension 1')
    axes[idx].set_ylabel('Dimension 2')
    axes[idx].grid(True, alpha=0.3)

plt.tight_layout()
plt.show()

## Convergence Comparison

Let's compare how MC and QMC converge for a simple integration problem:
$$\int_0^1 \int_0^1 x^2 + y^2 \, dx \, dy = \frac{2}{3}$$

In [ ]:
def test_function(x, y):
    return x**2 + y**2

# True value
true_value = 2/3

# Sample sizes to test
fib_indices = range(6, 15)
errors_glp = []
sample_sizes = []

for m in fib_indices:
    glp = generators.good_lattice_points(m)
    N = len(glp)
    sample_sizes.append(N)
    
    # Compute integral
    values = test_function(glp[:, 0], glp[:, 1])
    estimate = np.mean(values)
    error = abs(estimate - true_value)
    errors_glp.append(error)

# MC comparison
errors_mc = []
np.random.seed(42)
for N in sample_sizes:
    mc_points = np.random.rand(N, 2)
    values = test_function(mc_points[:, 0], mc_points[:, 1])
    estimate = np.mean(values)
    error = abs(estimate - true_value)
    errors_mc.append(error)

# Plot convergence
plt.figure(figsize=(10, 6))
plt.loglog(sample_sizes, errors_glp, 'o-', label='QMC (GLP)', linewidth=2)
plt.loglog(sample_sizes, errors_mc, 's-', label='MC (Random)', linewidth=2)

# Reference lines
N_ref = np.array([sample_sizes[0], sample_sizes[-1]])
plt.loglog(N_ref, errors_glp[0] * (N_ref / sample_sizes[0])**(-1), 
           '--', label='N^(-1)', alpha=0.5)
plt.loglog(N_ref, errors_mc[0] * (N_ref / sample_sizes[0])**(-0.5), 
           '--', label='N^(-0.5)', alpha=0.5)

plt.xlabel('Sample Size (N)')
plt.ylabel('Absolute Error')
plt.title('MC vs QMC Convergence')
plt.legend()
plt.grid(True, alpha=0.3)
plt.show()

print(f"QMC achieves O(N^-1) convergence vs MC's O(N^-0.5)!")